<a href="https://colab.research.google.com/github/GentaKanno/tyushutu_honban/blob/main/%E6%96%87%E5%AD%97%E6%8A%BD%E5%87%BA%E6%9C%AC%E7%95%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import os

data = np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0]])
for i in range(1, 47):
    txt_file = f"A1-{i}.txt"
    png_file = f"png/strokes {i}.png"

    if not os.path.exists(txt_file):
        print(f"⚠️ {txt_file} が見つかりません。スキップします。")
        continue

    # Initialize variables that might not be defined in every iteration
    aspect_ratio = np.nan
    aspect_ratiorotated = np.nan
    entropy_blackratio = np.nan
    ratio_topbottom = np.nan
    ratio_leftrigft = np.nan
    ratio_diag = np.nan
    entropy_distance = np.nan
    circ_std = np.nan
    dx = np.nan
    dy = np.nan
    ratioorentropy_partiallen = np.nan
    ratioorentropy_partial_volume = np.nan
    ratioorentropy_strokeslen = np.nan
    ratioorentropy_strokesvolume = np.nan

    #-------------

    # TXT読み込み
    df = pd.read_csv(txt_file, sep=',')
    df[['x','y','stroke_id']] = df[['x','y','stroke_id']].astype(float)

    # 元のバウンディングボックス
    x_min, x_max = df['x'].min(), df['x'].max()
    y_min, y_max = df['y'].min(), df['y'].max()
    w, h = x_max - x_min, y_max - y_min

    # 出力サイズ
    out_size = 256
    margin = 20  # 周囲に余白（任意で調整）

    # スケーリング係数（長辺を out_size - 2*margin に収める）
    scale = (out_size - 2*margin) / max(w, h)

    # 出力キャンバス
    img = np.ones((out_size, out_size, 3), dtype=np.uint8) * 255

    # ストロークごとに描画
    for sid in df['stroke_id'].unique():
        stroke = df[df['stroke_id'] == sid].reset_index(drop=True)
        for j in range(1, len(stroke)): # Changed 'i' to 'j' to avoid conflict with loop variable 'i'
            x0 = int((stroke.loc[j-1, 'x'] - x_min) * scale + margin + (out_size - 2*margin - w*scale)/2)
            y0 = int((stroke.loc[j-1, 'y'] - y_min) * scale + margin + (out_size - 2*margin - h*scale)/2)
            x1 = int((stroke.loc[j, 'x'] - x_min) * scale + margin + (out_size - 2*margin - w*scale)/2)
            y1 = int((stroke.loc[j, 'y'] - y_min) * scale + margin + (out_size - 2*margin - h*scale)/2)
            cv.line(img, (x0, y0), (x1, y1), (0,0,0), 2)

    # 保存
    cv.imwrite(f"A1-{i}.png", img)

    #---------------

    #外接矩形の縦横比
    # --- 画像読み込み（グレースケール） ---
    img = cv.imread(f"A1-{i}.png", cv.IMREAD_GRAYSCALE)

    # --- 黒い部分を前景（白）にするため反転 ---
    bin_img = cv.bitwise_not(img)

    # --- 完全な二値化（念のため） ---
    _, bin_img = cv.threshold(bin_img, 128, 255, cv.THRESH_BINARY)

    # --- 輪郭抽出（黒画素を対象に） ---
    contours, _ = cv.findContours(bin_img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # --- 外接矩形の統合 ---
    if len(contours) == 0:
        print("黒画素が見つかりません。")
    else:
        x_min = min([cv.boundingRect(c)[0] for c in contours])
        y_min = min([cv.boundingRect(c)[1] for c in contours])
        x_max = max([cv.boundingRect(c)[0] + cv.boundingRect(c)[2] for c in contours])
        y_max = max([cv.boundingRect(c)[1] + cv.boundingRect(c)[3] for c in contours])

        w, h = x_max - x_min, y_max - y_min
        aspect_ratio = h / w if w > 0 else 0


        print(f"縦横比（h/w）: {aspect_ratio:.3f}")

    #-------------------

    #45°外接矩形の縦横比

    # --- 画像サイズと中心 ---
    h, w = img.shape
    center = (w // 2, h // 2)

    # --- 反時計回りに45°回転 ---
    angle = 45
    rot_mat = cv.getRotationMatrix2D(center, angle, 1.0)

    # 出力画像サイズ（回転しても切れないように拡張）
    cos = np.abs(rot_mat[0, 0])
    sin = np.abs(rot_mat[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))

    # 回転中心を補正
    rot_mat[0, 2] += (new_w / 2) - center[0]
    rot_mat[1, 2] += (new_h / 2) - center[1]

    # --- 回転実行 ---
    rotated = cv.warpAffine(img, rot_mat, (new_w, new_h), flags=cv.INTER_LINEAR, borderValue=255)

    # --- 二値化（黒→白に反転） ---
    _, bin_img = cv.threshold(rotated, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # --- 輪郭抽出（黒画素領域） ---
    contours, _ = cv.findContours(bin_img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # --- 全ての輪郭の外接矩形を統合 ---
    x_min = min([cv.boundingRect(c)[0] for c in contours])
    y_min = min([cv.boundingRect(c)[1] for c in contours])
    x_max = max([cv.boundingRect(c)[0] + cv.boundingRect(c)[2] for c in contours])
    y_max = max([cv.boundingRect(c)[1] + cv.boundingRect(c)[3] for c in contours])

    # --- 外接矩形の幅・高さ・縦横比 ---
    W, H = x_max - x_min, y_max - y_min
    aspect_ratiorotated = H / W

    print(f"45°外接矩形の縦横比: {aspect_ratiorotated:.3f}")


    #------------

    #4分割黒画素比エントロピー
    from math import log2

    # --- 二値化（黒文字→白に反転）---
    _, bin_img = cv.threshold(img, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # --- 画像サイズ取得 ---
    h, w = bin_img.shape

    # --- 4分割（上左・上右・下左・下右） ---
    h_half, w_half = h // 2, w // 2
    regions = [
        bin_img[0:h_half, 0:w_half],      # 左上
        bin_img[0:h_half, w_half:w],      # 右上
        bin_img[h_half:h, 0:w_half],      # 左下
        bin_img[h_half:h, w_half:w],      # 右下
    ]

    # --- 各領域の黒画素率を計算 ---
    black_ratios = []
    for k, region in enumerate(regions): # Changed 'i' to 'k' to avoid conflict with loop variable 'i'
        black_pixels = np.sum(region > 0)      # 黒画素（255値）の数
        total_pixels = region.size
        ratio = black_pixels / total_pixels
        black_ratios.append(ratio)

    # --- エントロピーを計算 ---
    # ※ 0 を含むと log2(0) になるので、微小値を加える
    p = np.array(black_ratios)
    p = p / p.sum() if p.sum() > 0 else p  # 確率として正規化
    entropy_blackratio = -np.sum([x * log2(x) for x in p if x > 0])

    print(f"エントロピー: {entropy_blackratio:.4f}")

    #-------------

    #上下黒画素率比

    # --- 二値化（黒文字→白に反転）---
    _, bin_img = cv.threshold(img, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # --- 画像サイズ取得 ---
    h, w = bin_img.shape
    h_half = h // 2

    # --- 上下2分割 ---
    top = bin_img[0:h_half, :]
    bottom = bin_img[h_half:h, :]

    # --- 各領域の黒画素率を計算 ---
    def black_ratio(region):
        return np.sum(region > 0) / region.size

    top_ratio = black_ratio(top)
    bottom_ratio = black_ratio(bottom)

    # --- 黒画素率の比（上:下）を計算 ---
    # 下が0の場合のエラー防止
    ratio_topbottom = top_ratio / bottom_ratio if bottom_ratio > 0 else np.inf

    # --- 結果出力 ---
    print(f"黒画素率の比（上:下）= {ratio_topbottom:.4f}")

    #------------

    #左右黒画素率比
    # --- 画像読み込み（グレースケール） ---

    # --- 二値化（黒文字→白に反転）---
    _, bin_img = cv.threshold(img, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # --- 画像サイズ取得 ---
    h, w = bin_img.shape
    w_half = w // 2

    # --- 左右2分割 ---
    left = bin_img[:, 0:w_half]
    right = bin_img[:, w_half:w]

    # --- 各領域の黒画素率を計算 ---
    def black_ratio(region):
        return np.sum(region > 0) / region.size

    left_ratio = black_ratio(left)
    right_ratio = black_ratio(right)

    # --- 黒画素率の比（左:右）を計算 ---
    ratio_leftrigft = left_ratio / right_ratio if right_ratio > 0 else np.inf

    # --- 結果出力 ---
    print(f"黒画素率の比（左:右）= {ratio_leftrigft:.4f}")

    #------------

    #対角黒画素率比
    # --- 画像読み込み（グレースケール） ---

    # --- 二値化（黒文字→白に反転）---
    _, bin_img = cv.threshold(img, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # --- サイズ取得 ---
    h, w = bin_img.shape
    h_half, w_half = h // 2, w // 2

    # --- 4分割 ---
    q1 = bin_img[0:h_half, w_half:w]     # 第1象限（右上）
    q2 = bin_img[0:h_half, 0:w_half]     # 第2象限（左上）
    q3 = bin_img[h_half:h, 0:w_half]     # 第3象限（左下）
    q4 = bin_img[h_half:h, w_half:w]     # 第4象限（右下）

    # --- 黒画素率計算 ---
    def black_ratio(region):
        return np.sum(region > 0) / region.size

    r1, r2, r3, r4 = map(black_ratio, [q1, q2, q3, q4])

    # --- 対角比 (第1+第3) / (第2+第4) ---
    num = r1 + r3
    den = r2 + r4
    ratio_diag = num / den if den > 0 else np.inf

    # --- 結果出力 ---
    print(f"対角黒画素率比 (1+3) / (2+4) = {ratio_diag:.4f}")

    #------------

    #4分割重心距離エントロピー
    # --- 画像読み込み ---

    # --- 二値化（黒文字を白に反転）---
    _, bin_img = cv.threshold(img, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # --- サイズと中心 ---
    h, w = bin_img.shape
    h_half, w_half = h // 2, w // 2
    center = np.array([w / 2, h / 2])

    # --- 4分割（完全4等分版） ---
    q1 = bin_img[0:h_half, w_half:w]      # 第1象限（右上）
    q2 = bin_img[0:h_half, 0:w_half]      # 第2象限（左上）
    q3 = bin_img[h_half:h, 0:w_half]      # 第3象限（左下）
    q4 = bin_img[h_half:h, w_half:w]      # 第4象限（右下）
    quadrants = [q1, q2, q3, q4]

    # --- 各領域のオフセット（座標原点を画像全体座標系に合わせる） ---
    offsets = [
        (w_half, 0),     # q1
        (0, 0),          # q2
        (0, h_half),     # q3
        (w_half, h_half) # q4
    ]

    # --- 各領域の重心を求める ---
    centroids = []
    for q, (ox, oy) in zip(quadrants, offsets):
        ys, xs = np.where(q > 0)
        if len(xs) == 0:
            centroids.append(None)
            continue
        cx = np.mean(xs) + ox
        cy = np.mean(ys) + oy
        centroids.append(np.array([cx, cy]))

    # --- 画像中心からの距離を計算 ---
    distances = []
    for c in centroids:
        if c is not None:
            d = np.linalg.norm(c - center)
            distances.append(d)
        else:
            distances.append(0.0)  # 黒画素がない場合は0距離に

    # --- 正規化してエントロピーを算出 ---
    p = np.array(distances) / np.sum(distances)
    entropy_distance = -np.sum(p * np.log2(p + 1e-10))  # 0対策

    # --- 結果表示 ---
    print(f"\n距離のエントロピー: {entropy_distance:.4f}")

    #----------

    #4分割重心角度標準偏差

    # --- 二値化（黒文字→白に反転）---
    _, bin_img = cv.threshold(img, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # --- サイズと中心 ---
    h, w = bin_img.shape
    h_half, w_half = h // 2, w // 2
    center = np.array([w / 2, h / 2])

    # --- 4分割 ---
    q1 = bin_img[0:h_half, w_half:w]      # 第1象限（右上）
    q2 = bin_img[0:h_half, 0:w_half]      # 第2象限（左上）
    q3 = bin_img[h_half:h, 0:w_half]      # 第3象限（左下）
    q4 = bin_img[h_half:h, w_half:w]      # 第4象限（右下）
    quadrants = [q1, q2, q3, q4]

    # --- 各領域のオフセット ---
    offsets = [
        (w_half, 0),     # q1
        (0, 0),          # q2
        (0, h_half),     # q3
        (w_half, h_half) # q4
    ]

    # --- 各領域の重心 ---
    centroids = []
    for q, (ox, oy) in zip(quadrants, offsets):
        ys, xs = np.where(q > 0)
        if len(xs) == 0:
            centroids.append(None)
            continue
        cx = np.mean(xs) + ox
        cy = np.mean(ys) + oy
        centroids.append(np.array([cx, cy]))

    # --- 中心→重心ベクトルの角度を求める ---
    angles = []
    for c in centroids:
        if c is not None:
            dx_temp, dy_temp = c - center # Use temporary variables for dx, dy here to avoid overwriting outer loop's dx, dy too early
            angle = np.arctan2(-dy_temp, dx_temp)  # y軸下向き補正（画像座標系→通常座標系）
            angles.append(angle)
    angles = np.array(angles)

    # --- 円周統計: 標準偏差 ---
    mean_angle = np.arctan2(np.mean(np.sin(angles)), np.mean(np.cos(angles)))
    R = np.sqrt(np.mean(np.cos(angles))**2 + np.mean(np.sin(angles))**2)
    circ_std = np.sqrt(-2 * np.log(R))

    # --- 結果 ---
    print(f"円周標準偏差: {circ_std:.4f}")

    #----------

    #上下重心距離差

    # --- 二値化（黒文字を白に反転）---
    _, bin_img = cv.threshold(img, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # --- サイズ取得 ---
    h, w = bin_img.shape
    h_half = h // 2

    # --- 上下2分割 ---
    top = bin_img[0:h_half, :]
    bottom = bin_img[h_half:h, :]

    # --- 黒画素の重心を求める関数 ---
    def centroid(region, offset_y=0):
        ys, xs = np.where(region > 0)
        if len(xs) == 0:
            return None
        cx = np.mean(xs)
        cy = np.mean(ys) + offset_y
        return np.array([cx, cy])

    # --- 各領域の重心 ---
    top_c = centroid(top, 0)
    bottom_c = centroid(bottom, h_half)

    # --- x座標の差 ---
    if top_c is not None and bottom_c is not None:
        dx = top_c[0] - bottom_c[0]
    else:
        dx = np.nan

    print(f"x方向の差（上 - 下）: {dx:.2f}")

    #---------------

   #左右重心距離差

    # --- 二値化（黒文字→白に反転）---
    _, bin_img = cv.threshold(img, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # --- サイズ取得 ---
    h, w = bin_img.shape
    w_half = w // 2

    # --- 左右2分割 ---
    left = bin_img[:, 0:w_half]
    right = bin_img[:, w_half:w]

    # --- 黒画素の重心を求める関数 ---
    def centroid(region, offset_x=0):
        ys, xs = np.where(region > 0)
        if len(xs) == 0:
            return None
        cx = np.mean(xs) + offset_x
        cy = np.mean(ys)
        return np.array([cx, cy])

    # --- 各領域の重心 ---
    left_c = centroid(left, 0)
    right_c = centroid(right, w_half)

    # --- y座標の差 ---
    if left_c is not None and right_c is not None:
        dy = left_c[1] - right_c[1]
    else:
        dy = np.nan

    print(f"y方向の差（左 - 右）: {dy:.2f}")

    #------------

    #部分系長さ

    # --- 二値化（黒文字を白に反転）---
    _, bin_img = cv.threshold(img, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # --- 連結成分解析 ---
    num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(bin_img)

    # --- 背景(ラベル0)を除く ---
    areas = stats[1:, cv.CC_STAT_AREA]  # 1番目以降が文字部分

    #--------------

    #部分系長さの比やエントロピー

    # --- 二値化（黒文字を白に反転）---
    _, bin_img = cv.threshold(img, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # --- 連結成分解析 ---
    num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(bin_img)

    # --- 背景を除いた部分系の黒画素数 ---
    areas = stats[1:, cv.CC_STAT_AREA]  # 背景を除く
    n = len(areas)

    # --- 場合分け ---
    if n == 0:
        print("黒画素の連結成分が存在しません。")

    elif n == 1:
        print("部分系が1つのため、比・エントロピーはなし。")

    elif n == 2:
        ratioorentropy_partiallen = max(areas) / min(areas)
        print(f"部分系が2つ → 部分系長さの比 = {ratioorentropy_partiallen:.3f}")

    else:
        # --- エントロピーを手計算で求める ---
        p = areas / np.sum(areas)  # 各部分系の黒画素割合
        ratioorentropy_partiallen = -np.sum(p * np.log2(p + 1e-12))  # log(0)防止のため微小値を加算
        print(f"部分系が{n}個 → 部分系長さのエントロピー = {ratioorentropy_partiallen:.3f} bits")

    #-------------

    #部分系外接矩形面積

    # --- 二値化（黒文字を白に反転）---
    _, bin_img = cv.threshold(img, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # --- 連結成分解析 ---
    num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(bin_img)

    #---------------

    #部分系面積の比やエントロピー

    # --- 二値化（黒文字を白に反転）---
    _, bin_img = cv.threshold(img, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # --- 連結成分解析 ---
    num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(bin_img)

    # --- 背景を除いた外接矩形の面積 ---
    widths  = stats[1:, cv.CC_STAT_WIDTH]
    heights = stats[1:, cv.CC_STAT_HEIGHT]
    bbox_areas = widths * heights
    n = len(bbox_areas)

    # --- 場合分け ---
    if n == 0:
        print("黒画素の連結成分が存在しません。")

    elif n == 1:
        print("部分系が1つのため、比・エントロピーはなし。")

    elif n == 2:
        ratioorentropy_partial_volume = max(bbox_areas) / min(bbox_areas)
        print(f"部分系が2つ → 外接矩形面積比 = {ratioorentropy_partial_volume:.3f}")

    else:
        p = np.array(bbox_areas) / np.sum(bbox_areas)
        ratioorentropy_partial_volume = -np.sum(p * np.log2(p + 1e-12))
        print(f"部分系が{n}個 → 外接矩形面積分布のエントロピー = {ratioorentropy_partial_volume:.3f} bits")

    #--------------

    #ストロークの抽出
    # --- データ読み込み ---
    df = pd.read_csv(f"A1-{i}.txt")

    # --- stroke_idごとにグループ化 ---
    groups = df.groupby("stroke_id")


    #-------------

    #ストローク長さの比やエントロピー（時間順）

    # --- データ読み込み ---
    df = pd.read_csv(f"A1-{i}.txt")

    # --- stroke_idごとに画素数を算出 ---
    stroke_pixels = []
    out_size = 256
    margin = 20
    x_min, x_max = df["x"].min(), df["x"].max()
    y_min, y_max = df["y"].min(), df["y"].max()
    w, h = x_max - x_min, y_max - y_min
    scale = (out_size - 2*margin) / max(w, h)
    offset_x = margin + (out_size - 2*margin - w*scale)/2
    offset_y = margin + (out_size - 2*margin - h*scale)/2

    for sid in sorted(df["stroke_id"].unique()):
        stroke = df[df["stroke_id"] == sid].reset_index(drop=True)
        img = np.zeros((out_size, out_size), dtype=np.uint8)
        for k in range(1, len(stroke)): # Changed 'i' to 'k'
            x0 = int((stroke.loc[k-1, "x"] - x_min) * scale + offset_x)
            y0 = int((stroke.loc[k-1, "y"] - y_min) * scale + offset_y)
            x1 = int((stroke.loc[k, "x"] - x_min) * scale + offset_x)
            y1 = int((stroke.loc[k, "y"] - y_min) * scale + offset_y)
            cv.line(img, (x0, y0), (x1, y1), 255, 2)
        stroke_pixels.append(np.count_nonzero(img))

    # --- 場合分け ---
    n = len(stroke_pixels)

    if n == 0:
        print("ストロークが存在しません。")

    elif n == 1:
        print("ストロークが1本のため、比・エントロピーはなし。")

    elif n == 2:
        ratioorentropy_strokeslen = stroke_pixels[0] / stroke_pixels[1]
        print(f"ストロークが2本 → 画素数比（時間順）= {ratioorentropy_strokeslen:.3f}")

    else:
        p = np.array(stroke_pixels) / np.sum(stroke_pixels)
        ratioorentropy_strokeslen = -np.sum(p * np.log2(p + 1e-12))
        print(f"ストロークが{n}本 → ストローク画素数エントロピー = {ratioorentropy_strokeslen:.3f} bits")

    #-------------

    #ストローク面積の比やエントロピー（時間順）
    # --- ① TXT読み込み ---
    txt_file = f"A1-{i}.txt"
    df = pd.read_csv(txt_file, sep=',')
    df[['x','y','stroke_id']] = df[['x','y','stroke_id']].astype(float)

    # --- ② スケーリング・位置補正 ---
    x_min, x_max = df['x'].min(), df['x'].max()
    y_min, y_max = df['y'].min(), df['y'].max()
    w, h = x_max - x_min, y_max - y_min

    out_size = 256
    margin = 20
    scale = (out_size - 2*margin) / max(w, h)
    offset_x = margin + (out_size - 2*margin - w*scale)/2
    offset_y = margin + (out_size - 2*margin - h*scale)/2

    # --- ③ 各ストロークの外接矩形面積を計算 ---
    bbox_areas = []
    stroke_ids = sorted(df['stroke_id'].unique())  # 時間順（筆順）

    for sid in stroke_ids:
        stroke = df[df['stroke_id'] == sid].reset_index(drop=True)

        xs = (stroke['x'] - x_min) * scale + offset_x
        ys = (stroke['y'] - y_min) * scale + offset_y

        x_min_s, x_max_s = xs.min(), xs.max()
        y_min_s, y_max_s = ys.min(), ys.max()

        w_s = x_max_s - x_min_s
        h_s = y_max_s - y_min_s
        area = w_s * h_s
        bbox_areas.append(area)

    # --- ④ 結果の出力 ---
    n = len(bbox_areas)

    if n == 0:
        print("ストロークが存在しません。")

    elif n == 1:
        print("ストロークが1本のため、比・エントロピーはなし。")

    elif n == 2:
        ratioorentropy_strokesvolume = bbox_areas[0] / bbox_areas[1]
        print(f"ストロークが2本 → 外接矩形面積比（筆順）= {ratioorentropy_strokesvolume:.3f}")

    else:
        p = np.array(bbox_areas) / np.sum(bbox_areas)
        ratioorentropy_strokesvolume = -np.sum(p * np.log2(p + 1e-12))
        print(f"ストロークが{n}本 → ストローク外接矩形面積分布のエントロピー = {ratioorentropy_strokesvolume:.3f} bits")
    #----------
    dataplus = np.array([[aspect_ratio, aspect_ratiorotated, entropy_blackratio, ratio_topbottom, ratio_leftrigft, ratio_diag, entropy_distance, circ_std, dx, dy, ratioorentropy_partiallen, ratioorentropy_partial_volume, ratioorentropy_strokeslen, ratioorentropy_strokesvolume]])
    data = np.append(data, dataplus, axis=0)

縦横比（h/w）: 1.244
45°外接矩形の縦横比: 1.191
エントロピー: 1.9781
黒画素率の比（上:下）= 0.8195
黒画素率の比（左:右）= 1.3267
対角黒画素率比 (1+3) / (2+4) = 1.0852

距離のエントロピー: 1.9859
円周標準偏差: 2.3257
x方向の差（上 - 下）: 6.28
y方向の差（左 - 右）: -9.07
部分系が1つのため、比・エントロピーはなし。
部分系が1つのため、比・エントロピーはなし。
ストロークが3本 → ストローク画素数エントロピー = 1.272 bits
ストロークが3本 → ストローク外接矩形面積分布のエントロピー = 1.192 bits
縦横比（h/w）: 0.621
45°外接矩形の縦横比: 1.050
エントロピー: 1.5428
黒画素率の比（上:下）= 0.9401
黒画素率の比（左:右）= 2.6930
対角黒画素率比 (1+3) / (2+4) = 3.3402

距離のエントロピー: 1.9930
円周標準偏差: 2.4375
x方向の差（上 - 下）: 78.02
y方向の差（左 - 右）: 51.81
部分系が2つ → 部分系長さの比 = 2.693
部分系が2つ → 外接矩形面積比 = 4.305
ストロークが2本 → 画素数比（時間順）= 2.693
ストロークが2本 → 外接矩形面積比（筆順）= 4.771
縦横比（h/w）: 1.955
45°外接矩形の縦横比: 0.737
エントロピー: 1.8715
黒画素率の比（上:下）= 1.4283
黒画素率の比（左:右）= 0.8029
対角黒画素率比 (1+3) / (2+4) = 0.5000

距離のエントロピー: 1.9879
円周標準偏差: 2.0798
x方向の差（上 - 下）: -13.31
y方向の差（左 - 右）: -62.52
部分系が2つ → 部分系長さの比 = 4.698
部分系が2つ → 外接矩形面積比 = 11.286
ストロークが2本 → 画素数比（時間順）= 0.213
ストロークが2本 → 外接矩形面積比（筆順）= 0.080
縦横比（h/w）: 1.190
45°外接矩形の縦横比: 0.870
エントロピー: 1.6758
黒画素率の比（上:下）= 0.42

In [ ]:
np.savetxt("outnp.csv", data, delimiter=",")